# Modelos 3
### Uso de regressão L1 e L2
Nome: Joaquim Junior e Matheus Velloso

#### 1. Introdução
Em Machine Learning, a regressão linear é um dos modelos mais simples de previsão de dados. Porém, ela pode sofrer com o Underfit e Overfit. O que acontece é que a regressão linear faz previsões calculando um polinômio para tentar representar o resultado. Porém, para representar com grande acurácia os dados de treino, é necessário um polinômio de grau alto, que pode gerar resultados melhores para os dados de treino, mas piores para os demais, ou seja overfit (super ajustado). Veja na imagem abaixo:

<div align="center">
    <img src="https://analystprep.com/study-notes/wp-content/uploads/2021/03/Img_13.jpg" alt="Fern vs Ehre" width="800">
</div>

Pensando nisso, foram desenvolvidos modelos de regularização do grau desses polinômios para evitar o Overfit, como é o caso do L1 (LASSO) e L2 (Ridge), que serão estudados a seguir.

### 2. Teoria
Ambos os métodos tentam diminuir o efeito de Overfitting da regressão. Primeiramento relembremos o cálculo do MSE:

$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2
$$

Suponha que o valor a ser predito seja um polinômio tal que:
$$\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2^2 + \theta_3 x_3^3 + \theta_4 x_4^4...$$

Substituindo:
$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - (\theta_0 + \theta_1 x_1 + \theta_2 x_2^2 + \theta_3 x_3^3 + \theta_4 x_4^4...))^2
$$

Chamemos o polinômio de f, para simplificar
$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - f(x_i))^2
$$

Para diminuir o grau do polinômio, a regularização acrescenta um demérito para graus maiores. Veja como é feito em L2:

$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - f(x_i))^2 + \alpha\sum_{i=1}^{n}(\theta_i)^2
$$

Ou seja, quanto maior n, maior o somatório dos quadrados de theta, sendo vantajoso um modelo que tenha menor grau e boa precisão. Alpha é uma constante que pode ser escolhida para ditar o peso do demérito.

Agora, vejamos como é feito em L1:

$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - f(x_i))^2 + \alpha\sum_{i=1}^{n}|\theta_i|
$$

Ele também adiciona um demérito para o aumento de grau, mas agora usando o módulo de theta com a constante alpha.

Por fim, o Elastic-Net é o efeito de L1 e L2 juntos:

$$
MSE = \frac{1}{n}\sum_{i=1}^{n}(y_i - f(x_i))^2 + \alpha(l_1ratio)\sum_{i=1}^{n}|\theta_i| + 0.5\alpha(1-l_1ratio)\sum_{i=1}^{n}(\theta_i)^2
$$

### 3. Uso
#### 3.1 Importação e Tratamento do Dataframe
Usarei o data set student_exam_scores, disponível na plataforma Kaggle

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Importar e tratar dataset
df = pd.read_csv("student_exam_scores.csv")
df = df.iloc[0:, 1:]
df = df.dropna()
df

,hours_studied,sleep_hours,attendance_percent,previous_scores,exam_score
0,8.0,8.8,72.1,45,30.2
1,1.3,8.6,60.7,55,25.0
2,4.0,8.2,73.7,86,35.8
3,3.5,4.8,95.1,66,34.0
4,9.1,6.4,89.8,71,40.3
...,...,...,...,...,...
195,10.5,5.4,94.0,87,42.7
196,7.1,6.1,85.1,92,40.4
197,1.6,6.9,63.8,76,28.2
198,12.0,7.3,50.5,58,42.0


In [3]:
X = df.drop("exam_score", axis=1)
y = df["exam_score"]
SEMENTE = 27
#Divisão treino teste
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=SEMENTE)

#### 3.2 Regressão Linear

Façamos agora uma Regressão Linear

In [4]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(train_x, train_y)
print(f"O score nos casos de teste foram {reg.score(test_x, test_y)} enquanto no de treino {reg.score(train_x, train_y)}.")

O score nos casos de teste foram 0.7254087630089416 enquanto no de treino 0.8455916021138415.


O modelo obteve um score de 0.725 nos dados de teste. Tentaremos aplicar agora os regularizadores LASSO e Ridge e usarmos a regressão linear como padrão para compararmos suas eficiências.

#### 3.3 Regressão L1 (LASSO)
Façamos agora a regressão L1 para observarmos se de fato podemos aumentar a acuracia. Para esse exemplo usei alpha = 1

In [5]:
from sklearn import linear_model

lasso_reg = linear_model.Lasso(alpha=1)
lasso_reg.fit(train_x, train_y)

print(f"O score nos casos de teste foram {lasso_reg.score(test_x, test_y)} enquanto no de treino {lasso_reg.score(train_x, train_y)}.")

O score nos casos de teste foram 0.7609801969550539 enquanto no de treino 0.8341574289953586.


Veja que a acuracia aumentou significativamente, de 0.725 para 0.761 com baixa diferença nos casos de treino. Vejamos como varia a acurácia para diferentes alpha:

In [6]:
#Vejamos como varia com alpha:
scores_test = []
scores_train = []
for alp in [0.1, 1, 2, 5, 10, 50, 200]:
    lasso_reg = linear_model.Lasso(alpha=alp)
    lasso_reg.fit(train_x, train_y)
    scores_test.append(lasso_reg.score(test_x, test_y))
    scores_train.append(lasso_reg.score(train_x, train_y))
print(scores_test)
print(scores_train)

[0.7304482642755346, 0.7609801969550539, 0.7635921687691649, 0.7334644511650309, 0.5135731234012768, -0.3005492479741043, -0.3005492479741043]
[0.8454772481182318, 0.8341574289953586, 0.7998553275828353, 0.7442300774513069, 0.5768581614695175, 0.0, 0.0]


Portanto, o desempenho do algoritmo depende crucialmente do alpha escolhido, podendo ser visto que para os da lista, o com maior desempenho foi alpha = 1 ou 2

#### 3.4 Regressão L2 (RIDGE)
Farei agora a regressão L2 com um alpha = 200

In [7]:
ridge_reg = linear_model.Ridge(alpha=200)
ridge_reg.fit(train_x, train_y)
print(f"O score nos casos de teste foram {ridge_reg.score(test_x, test_y)} enquanto no de treino {ridge_reg.score(train_x, train_y)}.")

O score nos casos de teste foram 0.7615529516656562 enquanto no de treino 0.8357295946214666.


Veja que a acuracia também pôde aumentar de forma significativa usando o Ridge, de 0.725 para 0.761 com também baixa diferença nos casos de treino. Variemos alpha agora:

In [8]:
#Vejamos como varia com alpha:
scores_test = []
scores_train = []
for alp in [0.1, 1, 2, 5, 10, 50, 200]:
    lasso_reg = linear_model.Ridge(alpha=alp)
    lasso_reg.fit(train_x, train_y)
    scores_test.append(lasso_reg.score(test_x, test_y))
    scores_train.append(lasso_reg.score(train_x, train_y))
print(scores_test)
print(scores_train)

[0.725441519317793, 0.7257353569675498, 0.7260598074319422, 0.7270204555595647, 0.7285801114372805, 0.7393871256640926, 0.7615529516656562]
[0.8455915979718392, 0.8455911891806184, 0.8455899559760613, 0.8455814172622392, 0.8455515346532589, 0.8447066959900845, 0.8357295946214666]


É possível perceber que alpha varia de modo bem diferente em L1 e L2, mas para ambos apresentam grande diferença. Para L2, o melhor alpha foi 200.

#### 3.5 Regressão L1 e L2 (Elastic-Net)
Por fim, usaremos o Elastic-Net, que combina L1 e L2. Farei com alpha = 1 e divisão entre L1 e L2 de 0.7 para 0.3. 

In [9]:
en_reg = linear_model.ElasticNet(alpha=1, l1_ratio=0.6)
en_reg.fit(train_x, train_y)
print(f"O score nos casos de teste foram {en_reg.score(test_x, test_y)} enquanto no de treino {en_reg.score(train_x, train_y)}.")

O score nos casos de teste foram 0.7610673614360217 enquanto no de treino 0.8362475907608985.


Veja que foi possível, assim como em L1 e L2, melhorar significativamente a acuracia, porém com agora uma gama muito maior de opções para isso, já que combina ambos os modelos vistos. Vejamos como varia o score variando l1_ratio e alpha:

In [10]:
#Vejamos como varia com alpha:
scores_test = []
scores_train = []
best, rat, alph = 0, 0, 0
for ratio in [0.2, 0.4, 0.5, 0.6, 0.8]:
    for alp in [0.1, 1, 2, 5, 10, 50, 200]:
        lasso_reg = linear_model.ElasticNet(alpha=alp, l1_ratio=ratio)
        lasso_reg.fit(train_x, train_y)
        score = lasso_reg.score(test_x, test_y)
        scores_test.append(score)
        scores_train.append(lasso_reg.score(train_x, train_y))
        if score > best:
            best, rat, alph = score, ratio, alp

    print(f"Para l1_ratio={ratio}:")
    print(scores_test)
    print(scores_train)
print(f"O melhor desempenho obtido nos casos teste foi para l1_ratio = {rat} e alpha = {alp} resultando em score de {best}")

Para l1_ratio=0.2:
[0.7308697787905603, 0.7603734860296405, 0.7683518757142237, 0.7288368969972976, 0.6065087301991618, 0.054396902451969265, -0.26000739133353323]
[0.8454690703978525, 0.8371160678186716, 0.8204086173235896, 0.7582440415724203, 0.6569361042941235, 0.29038432658386426, 0.03391541039642132]
Para l1_ratio=0.4:
[0.7308697787905603, 0.7603734860296405, 0.7683518757142237, 0.7288368969972976, 0.6065087301991618, 0.054396902451969265, -0.26000739133353323, 0.7307668379672224, 0.7607948755950209, 0.7695377873931322, 0.7259577506872463, 0.5932028878739339, -0.14335186769390051, -0.3005492479741043]
[0.8454690703978525, 0.8371160678186716, 0.8204086173235896, 0.7582440415724203, 0.6569361042941235, 0.29038432658386426, 0.03391541039642132, 0.8454722660445787, 0.8367933398006434, 0.8184946060846876, 0.7468134198318727, 0.641620383863486, 0.15100701101451464, 0.0]
Para l1_ratio=0.5:
[0.7308697787905603, 0.7603734860296405, 0.7683518757142237, 0.7288368969972976, 0.6065087301991618

Portanto, de fato o Elastic-Net possui um grande potencial, tendo tido como melhor score para os casos teste em l1_ratio = 0.5 e alpha = 200, com o surpreendente score de aproximadamente 0.77.

### 4. Discussão
Ambos os modelos de regressão L1 e L2 se provaram muito eficazes em melhorar facilmente e significativamente a acuracia do modelo de regressão linear. Em comparação com a regressão linear tradicional, que obteve score de 0.72, obtivemos rapidamente score de 0.77 usando o Elastic-Net. A variação entre os alpha usados e no caso do Elastic-Net a proporção L1/L2 demonstraram alterar significativamente o score final, sendo importante portanto atenção e teste dos valores usados para maximizar a potência dos regularizadores.

### 5. Conclusão
Os modelos Lasso, Ridge e Elastic-Net se provaram ferramentas muito úteis para melhorar a acuracia de modelos de regressão linear, sendo ferramentas agora indispensáveis a elas.

### 6. Referências
DATACAMP. Tutorial: Regressão Lasso e Ridge no Python. 2025. Disponível em: https://www.datacamp.com/pt/tutorial/tutorial-lasso-ridge-regression. Acesso em: 27 set. 2025.

PEDREGOSA, F. et al. sklearn.linear_model.ElasticNet. Scikit-learn, 2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html. Acesso em: 27 set. 2025.

THE PANDAS DEVELOPMENT TEAM. pandas.DataFrame.drop. Pandas, 2025. Disponível em: https://pandas.pydata-org.translate.goog/docs/reference/api/pandas.DataFrame.drop.html?_x_tr_sl=en&_x_tr_tl=pt&_x_tr_hl=pt&_x_tr_pto=tc. Acesso em: 27 set. 2025.

ZINI, Fernando. Regressão Linear: Teoria e Prática. YouTube, 10 set. 2023. Disponível em: https://www.youtube.com/watch?v=VqKq78PVO9g. Acesso em: 27 set. 2025.

PEDREGOSA, F. et al. 1.1. Linear Models. Scikit-learn, 2025. Disponível em: https://scikit-learn.org/stable/modules/linear_model.html. Acesso em: 27 set. 2025.

DeepSeek. Disponível em: https://chat.deepseek.com/share/j7z37nakhld1e4jftp. Acesso em: 27 set. 2025.


KAGGLE. Student Academic Performance Trends Dataset. 2024. Disponível em: https://www.kaggle.com/datasets/emanfatima2025/student-academic-performance-trends